There are several approaches to using probabilistic modeling for classification tasks, e. g. Bayesian Neural Networks and Gaussian Process Models. Here, I decided to present Bayesian logistic regression as a means to solve a classification problem (please see http://edwardlib.org/iclr2017, https://github.com/blei-lab/edward/blob/master/examples/bayesian_logistic_regression.py, and http://edwardlib.org/tutorials/supervised-regression).

### Setup the system

In [1]:
import pandas     as pd
import edward     as ed
import numpy      as np
import tensorflow as tf

from edward.models import Bernoulli, Empirical, Normal

### Access the data

In [2]:
df_data = pd.read_csv('https://raw.githubusercontent.com/LEggert/Probabilistic-Programming/master/use_case_classification/use_case_classification_data.csv')

In [3]:
df_data.shape

(4521, 17)

In [4]:
df_data.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,30,unemployed,married,primary,no,1787,no,no,cellular,19,oct,79,1,-1,0,unknown,no
1,33,services,married,secondary,no,4789,yes,yes,cellular,11,may,220,1,339,4,failure,no
2,35,management,single,tertiary,no,1350,yes,no,cellular,16,apr,185,1,330,1,failure,no
3,30,management,married,tertiary,no,1476,yes,yes,unknown,3,jun,199,4,-1,0,unknown,no
4,59,blue-collar,married,secondary,no,0,yes,no,unknown,5,may,226,1,-1,0,unknown,no


Format target variable and all features as floats.

In [5]:
df_data = df_data.replace({'y': {'no': 0, 'yes': 1}})
df_data = df_data[['balance', 'duration', 'day', 'y']].astype(np.float64)

Let us assume for the purpose of this example that credit lending is based primarily on balance, day, and duration.

In [6]:
X_train = df_data[['balance', 'day', 'duration']].iloc[:200].values
y_train = df_data['y'].iloc[:200].values
X_test  = df_data[['balance', 'day', 'duration']].iloc[300:500].values
y_test  = df_data['y'].iloc[300:500].values

### Construct the model

In [7]:
n    = X_train.shape[0]  # number of observations
d    = X_train.shape[1]  # number of features

X = tf.placeholder(tf.float32, [n, d])
w = Normal(loc=tf.zeros(d), scale=3.0 * tf.ones(d))
b = Normal(loc=tf.zeros([]), scale=3.0 * tf.ones([]))
y = Bernoulli(logits=ed.dot(X, w) + b)

### Draw inference

In [8]:
t = 5000  # number of samples

qw = Empirical(params=tf.Variable(tf.random_normal([t, d])))
qb = Empirical(params=tf.Variable(tf.random_normal([t])))

inference = ed.HMC({w: qw, b: qb}, data={X: X_train, y: y_train})
inference.initialize(n_print=10, step_size=0.6)

tf.global_variables_initializer().run()

### Analyse the model

Form the posterior predictive distribution.

In [9]:
y_posterior = ed.copy(y, {w: qw, b: qb})

Calculate mean squared error.

In [10]:
ed.evaluate('mean_squared_error', data={X: X_test, y_posterior: y_test})

0.81